#### 개정 일자: 2020.05.02

1) 시작날짜, 종료날짜 입력하여 실행하도록 설정

2) column단위 병합에서 row 단위 개별 추가로 변경

In [1]:
import re
import time
import requests
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [2]:
#수집 : keyword(키워드, sortNo(정렬기준)
def searchNewsExtractor1(keyword,sortNo): #수동
    global soup
    pageNcnt = soup.find('div', {'class' : 'title_desc all_my'}).text
    match = re.search('\d*,*\d*건',pageNcnt)
    news_count = int(str(match.group())[:-1].replace(",",""))
    
    for i in range(news_count//10+1): #4,000건이 최대    
        pageNo= 10*i+1
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort="+str(sortNo)+"&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:da,p:all,a:all&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'lxml')
        newsParser()
        
        if req.status_code==200:
            print("뉴스 ",pageNo,"번째 수집 완료")

In [3]:
#수집 : keyword/ 정렬순서(최신/과거)/ 날짜 지정 
def searchNewsExtractor2(keyword,sortNo,sdate,edate): #자동
    global soup
    dot_sdate = sdate[0:4]+"."+sdate[4:6]+"."+sdate[6:8]
    dot_edate = edate[0:4]+"."+edate[4:6]+"."+edate[6:8]
    
    url1 = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort="+str(sortNo)+"&photo=0&field=0&reporter_article=&pd=3&ds="+dot_sdate+"&de="+dot_edate+"&docid=&nso=so:dd,p:from"+sdate+"to"+edate+"%2Ca%3Aall&mynews=0&start=1+&refresh_start=0"
    req1 = requests.get(url1)
    html1 = req1.text
    soup1 = BeautifulSoup(html1, 'lxml')
    
    pageNcnt = soup1.find('div', {'class' : 'title_desc all_my'}).text
    match = re.search('\d*,*\d*건',pageNcnt)
    news_count = int(str(match.group())[:-1].replace(",",""))
    
    for i in range(news_count//10+1): #4,000건이 최대
        pageNo= 10*i+1
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort="+str(sortNo)+"&photo=0&field=0&reporter_article=&pd=3&ds="+dot_sdate+"&de="+dot_edate+"&docid=&nso=so:dd,p:from"+sdate+"to"+edate+"%2Ca%3Aall&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'lxml')
        newsParser()
        
        if req.status_code==200:
            print(keyword,"/",sdate,"~",edate,"--> " ,pageNo,"/",news_count)
            #print(url)

In [4]:
def newsParser():
    global href_list
    global title_list
    global press_list
    global date_list
    global content_list

    #1. 링크, 2. 타이틀
    hrefNtitle = soup.find_all(class_=[" _sp_each_title","_sp_each_url _sp_each_title"])    
    for i in hrefNtitle:
        href_list.append(i['href']) 
        title_list.append(i['title'])
        
    #3. 언론사
    press = soup.find_all("span",{"class":"_sp_each_source"})
    for i in press:
        press_list.append(i.text)

    #4. 날짜(3가지 유형 - 날짜/일 전/ 시간 전)
    sourceNdate= soup.find_all("dd",{"class":"txt_inline"})

    for i in sourceNdate:
        try: 
            match = re.search(r'\d{4}.\d{2}.\d{2}', i.text)
            date_list.append(str(match.group()))
        except AttributeError:
            try: 
                if "일" in i.text:
                    match = re.search('\d+일 전', i.text)
                    date_list.append(str(match.group())) 
                elif "시간" in i.text:
                    match = re.search('\d+시간 전', i.text)
                    date_list.append(str(match.group()))
                else: pass
            except AttributeError: 
                date_list.append(i.text)
            
    # 5. 본문
    articles = soup.select('.type01 > li')
    for i in range(len(articles)):
        content_list.append(articles[i].select('dl > dd')[1].text)

In [5]:
def to_excel(filenm):
    global df1   
    df1 =  DataFrame({"date":date_list,
                      "press": press_list, 
                      "title": title_list,  
                      "content": content_list,
                      "url":href_list})
    
    for i in range(len(df1)):
        #print(df1.date[i])

        #최신날짜 "-일전/-시간 전" 정제하기 
        if "일" in df1.date[i]:
            match = re.search('\d+', df1.date[i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(days=diff)
            df1.date[i] = date.strftime("%Y.%m.%d")

        elif "시간" in df1.date[i]:
            match = re.search('\d+', df1.date[i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(hours=diff)
            
            df1.date[i] = date.strftime("%Y.%m.%d")
            #print("몇시간전? :",diff,"\n현재시각: ",now, "\n바뀐 시간: ", date)
            #print("입력시간: ",date.strftime("%Y.%m.%d"))
        else: pass
    
    df1.to_excel(filenm, index=True)
    return df1

In [7]:
#####-------------main-----------------#####
href_list = []
title_list = []
press_list = []
date_list = []
content_list = []

keyword = "스트래티지애널리틱스" #카운터포인트리서치/스트래티지애널리틱스
sortNo = 1 #1:최신순, 2: 과거순
sdate = "20200801" #시작 날짜
edate = "20200831" #종료 날짜 
filenm = keyword+"_"+sdate+"_"+edate+".xlsx" #저장할 파일명

#1. 최신순/과거순
#searchNewsExtractor1(keyword,sortNo)

#2. 날짜지정
searchNewsExtractor2(keyword,sortNo,sdate,edate)
to_excel(filenm)

스트래티지애널리틱스 / 20200801 ~ 20200831 -->  1 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  11 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  21 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  31 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  41 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  51 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  61 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  71 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  81 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  91 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  101 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  111 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  121 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  131 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  141 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  151 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  161 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  171 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  181 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  191 / 239
스트래티지애널리틱스 / 20200801 ~ 20200831 -->  201 / 239
스트래

,date,press,title,content,url
0,2020.09.01,아이뉴스24언론사 선정,"삼성전자, 아재폰 오명 벗기 총력전…스마트폰에 활력 불어넣나",스트래티지애널리틱스(SA)에 따르면 상반기 삼성전자의 세계 스마트폰 시장점유율은 2...,http://www.inews24.com/view/1295098
1,2020.09.01,한국경제,"현대중공업, 서비스 로봇 시장 본격 진출…'비대면 산업' 공략",시장조사기관 스트래티지 애널리틱스(SA)에 따르면 세계 서비스 로봇 시장 규모는 지...,https://www.hankyung.com/economy/article/20200...
2,2020.09.01,데일리안,보급형 ‘갤S20 FE’ 국내 깜짝 등장...출시 임박,시장조사업체 스트래티지 애널리틱스(SA)에 따르면 삼성전자는 2분기 글로벌 스마트폰...,https://www.dailian.co.kr/news/view/915994/?sc...
3,2020.09.01,중앙일보,"저가폰에도 초고용량 배터리, 갤럭시M51 독일서 첫 출시",시장조사업체 스트래티지애널리틱스에 따르면 올 2분기(4~6월) 전 세계 스마트폰 시...,https://news.joins.com/article/olink/23455302
4,2020.09.01,데일리그리드,코로나 확산에 따른 거리두기로 AR·VR 콘텐츠 증가,글로벌 시장조사업체 스트래티지 애널리틱스(SA)는 VR·AR시장의 회복과 성장에 코...,http://www.dailygrid.net/news/articleView.html...
...,...,...,...,...,...
234,2020.08.03,디지털데일리언론사 선정,"삼성·화웨이·애플, 스마트폰 1위 ‘엎치락뒤치락’…2020년, 최후의 승자는?",시장조사기관 스트래티지애널리틱스(SA)는 지난 1분기 스마트폰 시장을 2억7480만...,http://www.ddaily.co.kr/news/article.html?no=1...
235,2020.08.03,테크M,[갤노트 크로니클②] 더 커진 스마트폰 세계…갤노트 '패블릿' 전성시대를 열다,시장조사기관 스트래티지애널리틱스에 따르면 2013년 삼성전자는 사상 최초로 3억대 ...,https://www.techm.kr/news/articleView.html?idx...
236,2020.08.02,디지털타임스언론사 선정,"폴더블폰 라인업 늘리는 삼성, 화웨이와 다시한번 맞붙는다",스트래티지 애널리틱스(SA)는 전 세계 폴더블폰 출하량이 2021년 3040만대에서...,http://www.dt.co.kr/contents.html?article_no=2...
237,2020.08.02,한스경제,"[갤럭시 언팩 미리보기] ②'절치부심' 노태문, 혁신으로 자존심 회복","업계에서는 그 중 SA(스트래티지애널리틱스)의 통계를 가장 신뢰하는데, 이 집계는 ...",http://www.sporbiz.co.kr/news/articleView.html...
